## 1) set config to be able to install packages

In [1]:
%%configure -f
{
    "conf": {
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
    }
}

In [2]:
from py4j.java_gateway import java_import

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1646053228230_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 2) install the needed package

In [3]:
sc.install_pypi_package("boto3")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


You are using pip version 9.0.1, however version 22.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [4]:
# Disable auto broadcasting to avoid a BroadcastHashJoin and a BroadcastExchange
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3) import the installed packages

In [5]:
import boto3
from pyspark.sql.functions import *
java_import(spark._sc._jvm, "org.apache.spark.sql.api.python.*")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 4) Data Exploration

### a) Read non-bucketed tables from parquet file

In [6]:
# The result of loading a parquet file is also a DataFrame.
fact_dk_earnings_original = spark.read.parquet("s3://die-stg-733963962829-eu-central-1/linh_thesis/fact_dk_earnings")
fact_leadouts_original = spark.read.parquet("s3://die-stg-733963962829-eu-central-1/linh_thesis/fact_leadouts")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
fact_leadouts_original.createOrReplaceTempView("fact_leadouts_original")
fact_dk_earnings_original.createOrReplaceTempView("fact_dk_earnings_original")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### b) Read data from spark Catalog

In [8]:
spark.catalog.listTables()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Table(name='bot_lookup', database='default', description=None, tableType='EXTERNAL', isTemporary=False), Table(name='bucketed_fact_dk_earnings', database='default', description=None, tableType='EXTERNAL', isTemporary=False), Table(name='bucketed_fact_leadouts', database='default', description=None, tableType='EXTERNAL', isTemporary=False), Table(name='bucketed_fact_leadouts_5_buckets', database='default', description=None, tableType='EXTERNAL', isTemporary=False), Table(name='bucketed_fact_leadouts_msid', database='default', description=None, tableType='EXTERNAL', isTemporary=False), Table(name='collect_traces', database='default', description=None, tableType='EXTERNAL', isTemporary=False), Table(name='die_hudi_isg_ro', database='default', description=None, tableType='EXTERNAL', isTemporary=False), Table(name='die_hudi_isg_rt', database='default', description=None, tableType='EXTERNAL', isTemporary=False), Table(name='die_hudi_tut', database='default', description=None, tableType='EXT

#### The result of the line above contains other tables of idealo in the Spark catalog as well. The important information is: 

Table(name='fact_leadouts_original', database=None, description=None, tableType='TEMPORARY', isTemporary=True)

Table(name='fact_dk_earnings_original', database=None, description=None, tableType='TEMPORARY', isTemporary=True)

Table(name='bucketed_fact_leadouts', database='default', description=None, tableType='EXTERNAL', isTemporary=False)

Table(name='bucketed_fact_dk_earnings', database='default', description=None, tableType='EXTERNAL', isTemporary=False)

Table(name='bucketed_fact_leadouts_5_buckets', database='default', description=None, tableType='EXTERNAL', isTemporary=False)

Table(name='bucketed_fact_leadouts_msid', database='default', description=None, tableType='EXTERNAL', isTemporary=False)

In [9]:
spark.sql("DESCRIBE TABLE EXTENDED bucketed_fact_leadouts").show(n=100, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+-------------------------------------------------------------------------+-------+
|col_name                      |data_type                                                                |comment|
+------------------------------+-------------------------------------------------------------------------+-------+
|main_session_id               |bigint                                                                   |null   |
|session_id                    |string                                                                   |null   |
|cookie_value                  |string                                                                   |null   |
|category_id                   |string                                                                   |null   |
|product_id                    |string                                                                   |null   |
|category_path_ids             |array<string>                                   

In [10]:
spark.sql("DESCRIBE TABLE EXTENDED bucketed_fact_dk_earnings").show(n=100, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+----------------------------------------------------------------------------+-------+
|col_name                      |data_type                                                                   |comment|
+------------------------------+----------------------------------------------------------------------------+-------+
|shop_id                       |string                                                                      |null   |
|category_id                   |string                                                                      |null   |
|category_path_ids             |int                                                                         |null   |
|order_hash_id                 |string                                                                      |null   |
|revenue                       |double                                                                      |null   |
|quantity                      |bigint                  

In [11]:
spark.sql("DESCRIBE TABLE EXTENDED bucketed_fact_leadouts_5_buckets").show(n=100, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+-----------------------------------------------------------------------------------+-------+
|col_name                      |data_type                                                                          |comment|
+------------------------------+-----------------------------------------------------------------------------------+-------+
|main_session_id               |bigint                                                                             |null   |
|session_id                    |string                                                                             |null   |
|cookie_value                  |string                                                                             |null   |
|category_id                   |string                                                                             |null   |
|product_id                    |string                                                                             |null   |


In [12]:
spark.sql("DESCRIBE TABLE EXTENDED bucketed_fact_leadouts_msid").show(n=100, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+------------------------------------------------------------------------------+-------+
|col_name                      |data_type                                                                     |comment|
+------------------------------+------------------------------------------------------------------------------+-------+
|main_session_id               |bigint                                                                        |null   |
|session_id                    |string                                                                        |null   |
|cookie_value                  |string                                                                        |null   |
|category_id                   |string                                                                        |null   |
|product_id                    |string                                                                        |null   |
|category_path_ids             |array<st

In [13]:
spark.sql("DESCRIBE TABLE EXTENDED fact_leadouts_original").show(n=100, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+-----------------------------------------+-------+
|col_name                      |data_type                                |comment|
+------------------------------+-----------------------------------------+-------+
|main_session_id               |bigint                                   |null   |
|session_id                    |string                                   |null   |
|cookie_value                  |string                                   |null   |
|category_id                   |string                                   |null   |
|product_id                    |string                                   |null   |
|category_path_ids             |array<string>                            |null   |
|media                         |string                                   |null   |
|site                          |string                                   |null   |
|site_type                     |string                                   |null   |
|cou

In [14]:
spark.sql("DESCRIBE TABLE EXTENDED fact_dk_earnings_original").show(n=100, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+---------+-------+
|col_name                      |data_type|comment|
+------------------------------+---------+-------+
|shop_id                       |string   |null   |
|category_id                   |string   |null   |
|category_path_ids             |int      |null   |
|order_hash_id                 |string   |null   |
|revenue                       |double   |null   |
|quantity                      |bigint   |null   |
|status                        |string   |null   |
|order_timestamp               |timestamp|null   |
|main_session_id               |bigint   |null   |
|session_id                    |string   |null   |
|cookie_value                  |string   |null   |
|product_id                    |string   |null   |
|media                         |string   |null   |
|site                          |string   |null   |
|site_type                     |string   |null   |
|country                       |string   |null   |
|brand                         

In [15]:
## non-bucketed tables
t1 = spark.table("fact_leadouts_original")
t2 = spark.table("fact_dk_earnings_original")

# bucketed tables
t3 = spark.table("bucketed_fact_leadouts")
t4 = spark.table("bucketed_fact_dk_earnings")
t5 = spark.table("bucketed_fact_leadouts_5_buckets")
t6 = spark.table("bucketed_fact_leadouts_msid")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
t1.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

53396574

In [17]:
t2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

599181

In [18]:
t3.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

53396574

In [19]:
t4.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

599181

In [20]:
t5.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

53396574

In [21]:
t6.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

53396574

## 6) Tests: Queries 

Note: the joins between fact_leadouts and fact_dk_earnings always contain shuffle, regardless of the join keys

In [22]:
query1 = spark.sql("SELECT t1.product_id, t1.category_id, t1.date_utc\
           FROM  fact_leadouts_original t1 join fact_dk_earnings_original t2 \
           ON t1.cookie_value = t2.cookie_value \
           WHERE t1.date_utc BETWEEN date('2021-12-01') AND date('2021-12-07') \
           and t2.date_utc BETWEEN date('2021-12-01') AND date('2021-12-07')")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
query1.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------+----------+
|product_id|category_id|  date_utc|
+----------+-----------+----------+
| 201168334|      21177|2021-12-05|
|   5680051|       3972|2021-12-02|
|   5680051|       3972|2021-12-02|
|   5680051|       3972|2021-12-02|
|   5680051|       3972|2021-12-02|
|   5680051|       3972|2021-12-02|
|   5680051|       3972|2021-12-02|
| 200364800|       9552|2021-12-04|
| 201609046|      18817|2021-12-03|
|   6395743|       3236|2021-12-05|
|   5102579|       3236|2021-12-05|
|   5135206|       3236|2021-12-05|
|   5135206|       3236|2021-12-05|
|   6395743|       3236|2021-12-05|
|   6395743|       3236|2021-12-05|
|   6395743|       3236|2021-12-05|
|   6395743|       3236|2021-12-05|
|   6395743|       3236|2021-12-05|
|   6395743|       3236|2021-12-05|
|   6395743|       3236|2021-12-05|
+----------+-----------+----------+
only showing top 20 rows

In [24]:
query1.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (13)
+- Project (12)
   +- SortMergeJoin Inner (11)
      :- Sort (5)
      :  +- Exchange (4)
      :     +- Project (3)
      :        +- Filter (2)
      :           +- Scan parquet  (1)
      +- Sort (10)
         +- Exchange (9)
            +- Project (8)
               +- Filter (7)
                  +- Scan parquet  (6)


(1) Scan parquet 
Output [5]: [cookie_value#62, category_id#63, product_id#64, date_utc#109, hour#110]
Batched: true
Location: InMemoryFileIndex [s3://die-stg-733963962829-eu-central-1/linh_thesis/fact_leadouts]
PartitionFilters: [isnotnull(date_utc#109), (date_utc#109 >= 18962), (date_utc#109 <= 18968)]
PushedFilters: [IsNotNull(cookie_value)]
ReadSchema: struct<cookie_value:string,category_id:string,product_id:string>

(2) Filter
Input [5]: [cookie_value#62, category_id#63, product_id#64, date_utc#109, hour#110]
Condition : (isnotnull(cookie_value#62) AND bloomfilter#1122 of [bf1122 cookie_value#10 estimatedNumRows=159220

In [25]:
query2 = spark.sql("SELECT t1.product_id, t1.category_id, t1.date_utc\
           FROM  fact_leadouts_original t1 join fact_dk_earnings_original t2 \
           ON t1.leadout_click_bokey = t2.leadout_click_bokey \
           WHERE t1.date_utc BETWEEN date('2021-12-01') AND date('2021-12-07') \
           and t2.date_utc BETWEEN date('2021-12-01') AND date('2021-12-07')")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
query2.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------+----------+
|product_id|category_id|  date_utc|
+----------+-----------+----------+
|   6826083|       2021|2021-12-05|
| 201130660|       1941|2021-12-05|
| 200521858|       5414|2021-12-05|
| 201682626|      30990|2021-12-03|
|   6568620|       3309|2021-12-03|
|   4791210|       3750|2021-12-07|
| 200578608|      32482|2021-12-07|
|    589125|       3972|2021-12-03|
|   5006663|      22699|2021-12-03|
| 201383628|      32530|2021-12-01|
|   3041390|       7174|2021-12-02|
|   6864854|      19116|2021-12-03|
|   5031158|      27733|2021-12-07|
| 200741011|       7734|2021-12-01|
| 200612395|       3747|2021-12-06|
|   1159285|      18423|2021-12-02|
|   1159285|      18423|2021-12-02|
| 201264818|       3751|2021-12-06|
|   6181718|      10832|2021-12-01|
| 201392308|      22506|2021-12-05|
+----------+-----------+----------+
only showing top 20 rows

In [27]:
query2.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (13)
+- Project (12)
   +- SortMergeJoin Inner (11)
      :- Sort (5)
      :  +- Exchange (4)
      :     +- Project (3)
      :        +- Filter (2)
      :           +- Scan parquet  (1)
      +- Sort (10)
         +- Exchange (9)
            +- Project (8)
               +- Filter (7)
                  +- Scan parquet  (6)


(1) Scan parquet 
Output [5]: [category_id#63, product_id#64, leadout_click_bokey#82, date_utc#109, hour#110]
Batched: true
Location: InMemoryFileIndex [s3://die-stg-733963962829-eu-central-1/linh_thesis/fact_leadouts]
PartitionFilters: [isnotnull(date_utc#109), (date_utc#109 >= 18962), (date_utc#109 <= 18968)]
PushedFilters: [IsNotNull(leadout_click_bokey)]
ReadSchema: struct<category_id:string,product_id:string,leadout_click_bokey:string>

(2) Filter
Input [5]: [category_id#63, product_id#64, leadout_click_bokey#82, date_utc#109, hour#110]
Condition : (isnotnull(leadout_click_bokey#82) AND bloomfilter#1140 of [bf1140 lead

### a) Join a bucketed table with a non-bucketed table on the bucket key

In [28]:
query3 = spark.sql("SELECT t1.product_id, t1.category_id, t1.date_utc\
           FROM  bucketed_fact_leadouts t1 join fact_dk_earnings_original t2 \
           ON t1.cookie_value = t2.cookie_value \
           WHERE t1.date_utc BETWEEN date('2021-12-01') AND date('2021-12-07') \
           and t2.date_utc BETWEEN date('2021-12-01') AND date('2021-12-07')")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
query3.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------+----------+
|product_id|category_id|  date_utc|
+----------+-----------+----------+
| 201581052|       4734|2021-12-06|
| 201581052|       4734|2021-12-06|
| 201581052|       4734|2021-12-06|
|   6795386|       2461|2021-12-02|
| 201436879|       9553|2021-12-05|
| 201436879|       9553|2021-12-05|
| 201436879|       9553|2021-12-05|
| 201436879|       9553|2021-12-05|
| 201436879|       9553|2021-12-05|
|    967230|       3972|2021-12-07|
|    967230|       3972|2021-12-07|
|    967230|       3972|2021-12-07|
|    600529|       8694|2021-12-05|
|   7061492|       8694|2021-12-04|
|   7061492|       8694|2021-12-06|
|   7061492|       8694|2021-12-06|
|   6725776|      11595|2021-12-01|
|   6725776|      11595|2021-12-01|
| 200838608|      18817|2021-12-01|
| 200867399|       2925|2021-12-03|
+----------+-----------+----------+
only showing top 20 rows

In [30]:
query3.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (13)
+- Project (12)
   +- SortMergeJoin Inner (11)
      :- Sort (5)
      :  +- Exchange (4)
      :     +- Project (3)
      :        +- Filter (2)
      :           +- Scan parquet default.bucketed_fact_leadouts (1)
      +- Sort (10)
         +- Exchange (9)
            +- Project (8)
               +- Filter (7)
                  +- Scan parquet  (6)


(1) Scan parquet default.bucketed_fact_leadouts
Output [5]: [cookie_value#447, category_id#448, product_id#449, date_utc#494, hour#495]
Batched: true
Location: InMemoryFileIndex [s3://die-stg-733963962829-eu-central-1/linh_thesis/bucketed_fact_leadouts/date_utc=2021-12-02/hour=14, ... 167 entries]
PartitionFilters: [isnotnull(date_utc#494), (date_utc#494 >= 18962), (date_utc#494 <= 18968)]
PushedFilters: [IsNotNull(cookie_value)]
ReadSchema: struct<cookie_value:string,category_id:string,product_id:string>

(2) Filter
Input [5]: [cookie_value#447, category_id#448, product_id#449, date_utc#494, h

### b) Join two bucketed tables with different numbers of buckets on the bucket key

In [31]:
query4 = spark.sql("SELECT t1.product_id, t1.category_id, t1.date_utc\
           FROM  bucketed_fact_leadouts_5_buckets t1 join bucketed_fact_dk_earnings t2 \
           ON t1.cookie_value = t2.cookie_value \
           WHERE t1.date_utc BETWEEN date('2021-12-01') AND date('2021-12-07') \
           and t2.date_utc BETWEEN date('2021-12-01') AND date('2021-12-07')")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
query4.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------+----------+
|product_id|category_id|  date_utc|
+----------+-----------+----------+
|   6795386|       2461|2021-12-02|
| 201581052|       4734|2021-12-06|
| 201581052|       4734|2021-12-06|
| 201581052|       4734|2021-12-06|
|   5486902|      27600|2021-12-07|
|   6450995|      18852|2021-12-06|
|   6450995|      18852|2021-12-06|
|   6527533|      18852|2021-12-06|
| 200946898|       3406|2021-12-05|
|   5744665|       3406|2021-12-05|
| 200788585|       3099|2021-12-05|
| 200788585|       3099|2021-12-05|
|   6586836|       4892|2021-12-07|
|   6586836|       4892|2021-12-07|
|   6586836|       4892|2021-12-07|
|   6586836|       4892|2021-12-07|
|   6586836|       4892|2021-12-07|
|   6586836|       4892|2021-12-07|
| 200589169|      19116|2021-12-05|
| 201136967|      21177|2021-12-06|
+----------+-----------+----------+
only showing top 20 rows

In [33]:
query4.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (12)
+- Project (11)
   +- SortMergeJoin Inner (10)
      :- Sort (4)
      :  +- Project (3)
      :     +- Filter (2)
      :        +- Scan parquet default.bucketed_fact_leadouts_5_buckets (1)
      +- Sort (9)
         +- Exchange (8)
            +- Project (7)
               +- Filter (6)
                  +- Scan parquet default.bucketed_fact_dk_earnings (5)


(1) Scan parquet default.bucketed_fact_leadouts_5_buckets
Output [5]: [cookie_value#609, category_id#610, product_id#611, date_utc#656, hour#657]
Batched: true
Location: InMemoryFileIndex [s3://die-stg-733963962829-eu-central-1/linh_thesis/bucketed_fact_leadouts_5_buckets/date_utc=2021-12-01/hour=3, ... 167 entries]
PartitionFilters: [isnotnull(date_utc#656), (date_utc#656 >= 18962), (date_utc#656 <= 18968)]
PushedFilters: [IsNotNull(cookie_value)]
ReadSchema: struct<cookie_value:string,category_id:string,product_id:string>
SelectedBucketsCount: 5 out of 5

(2) Filter
Input [5]: [cookie

### c) Join two bucketed tables with the same number of buckets on the bucketed key

In [34]:
query5 = spark.sql("SELECT t1.product_id, t1.category_id, t1.date_utc\
           FROM  bucketed_fact_leadouts t1 join bucketed_fact_dk_earnings t2 \
           ON t1.cookie_value = t2.cookie_value \
           WHERE t1.date_utc BETWEEN date('2021-12-01') AND date('2021-12-07') \
           and t2.date_utc BETWEEN date('2021-12-01') AND date('2021-12-07')")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
query5.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------+----------+
|product_id|category_id|  date_utc|
+----------+-----------+----------+
|   6795386|       2461|2021-12-02|
| 201581052|       4734|2021-12-06|
| 201581052|       4734|2021-12-06|
| 201581052|       4734|2021-12-06|
| 201617428|      25496|2021-12-06|
| 201617428|      25496|2021-12-06|
| 201617428|      25496|2021-12-06|
| 201617428|      25496|2021-12-06|
| 200349202|      15555|2021-12-05|
| 200349202|      15555|2021-12-05|
| 200349202|      15555|2021-12-05|
| 200349202|      15555|2021-12-05|
| 200757450|       2021|2021-12-04|
| 200757450|       2021|2021-12-04|
|   3476667|       9552|2021-12-04|
|   3476667|       9552|2021-12-04|
| 200949094|      12892|2021-12-04|
| 200949094|      12892|2021-12-04|
| 201134383|      19144|2021-12-04|
| 201134383|      19144|2021-12-04|
+----------+-----------+----------+
only showing top 20 rows

In [36]:
query5.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (11)
+- Project (10)
   +- SortMergeJoin Inner (9)
      :- Sort (4)
      :  +- Project (3)
      :     +- Filter (2)
      :        +- Scan parquet default.bucketed_fact_leadouts (1)
      +- Sort (8)
         +- Project (7)
            +- Filter (6)
               +- Scan parquet default.bucketed_fact_dk_earnings (5)


(1) Scan parquet default.bucketed_fact_leadouts
Output [5]: [cookie_value#447, category_id#448, product_id#449, date_utc#494, hour#495]
Batched: true
Location: InMemoryFileIndex [s3://die-stg-733963962829-eu-central-1/linh_thesis/bucketed_fact_leadouts/date_utc=2021-12-02/hour=14, ... 167 entries]
PartitionFilters: [isnotnull(date_utc#494), (date_utc#494 >= 18962), (date_utc#494 <= 18968)]
PushedFilters: [IsNotNull(cookie_value)]
ReadSchema: struct<cookie_value:string,category_id:string,product_id:string>
SelectedBucketsCount: 4 out of 4

(2) Filter
Input [5]: [cookie_value#447, category_id#448, product_id#449, date_utc#494, hour#

### d) Join two bucketed tables with the same number of buckets and bucket key but not on the bucket key

In [37]:
query6 = spark.sql("SELECT t1.product_id, t1.category_id, t1.date_utc\
           FROM  bucketed_fact_leadouts t1 join bucketed_fact_dk_earnings t2 \
           ON t1.leadout_click_bokey = t2.leadout_click_bokey \
           WHERE t1.date_utc BETWEEN date('2021-12-01') AND date('2021-12-07') \
           and t2.date_utc BETWEEN date('2021-12-01') AND date('2021-12-07')")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
query6.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------+----------+
|product_id|category_id|  date_utc|
+----------+-----------+----------+
|   6826083|       2021|2021-12-05|
| 201130660|       1941|2021-12-05|
| 200521858|       5414|2021-12-05|
| 201682626|      30990|2021-12-03|
|   6568620|       3309|2021-12-03|
|   4791210|       3750|2021-12-07|
| 200578608|      32482|2021-12-07|
|    589125|       3972|2021-12-03|
|   5006663|      22699|2021-12-03|
| 201383628|      32530|2021-12-01|
|   3041390|       7174|2021-12-02|
|   6864854|      19116|2021-12-03|
|   5031158|      27733|2021-12-07|
| 200741011|       7734|2021-12-01|
| 200612395|       3747|2021-12-06|
|   1159285|      18423|2021-12-02|
|   1159285|      18423|2021-12-02|
| 201264818|       3751|2021-12-06|
|   6181718|      10832|2021-12-01|
| 201392308|      22506|2021-12-05|
+----------+-----------+----------+
only showing top 20 rows

In [39]:
query6.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (13)
+- Project (12)
   +- SortMergeJoin Inner (11)
      :- Sort (5)
      :  +- Exchange (4)
      :     +- Project (3)
      :        +- Filter (2)
      :           +- Scan parquet default.bucketed_fact_leadouts (1)
      +- Sort (10)
         +- Exchange (9)
            +- Project (8)
               +- Filter (7)
                  +- Scan parquet default.bucketed_fact_dk_earnings (6)


(1) Scan parquet default.bucketed_fact_leadouts
Output [5]: [category_id#448, product_id#449, leadout_click_bokey#467, date_utc#494, hour#495]
Batched: true
Location: InMemoryFileIndex [s3://die-stg-733963962829-eu-central-1/linh_thesis/bucketed_fact_leadouts/date_utc=2021-12-02/hour=14, ... 167 entries]
PartitionFilters: [isnotnull(date_utc#494), (date_utc#494 >= 18962), (date_utc#494 <= 18968)]
PushedFilters: [IsNotNull(leadout_click_bokey)]
ReadSchema: struct<category_id:string,product_id:string,leadout_click_bokey:string>

(2) Filter
Input [5]: [category_id#

### e) Join two bucketed tables, which are bucketed with different bucket key


In [40]:
query7 = spark.sql("SELECT t1.product_id, t1.category_id, t1.date_utc\
           FROM  bucketed_fact_leadouts_msid t1 join bucketed_fact_dk_earnings t2 \
           ON t1.cookie_value = t2.cookie_value and t1.date_utc = t2.date_utc \
           WHERE t1.date_utc BETWEEN date('2021-12-01') AND date('2021-12-07') \
           and t2.date_utc BETWEEN date('2021-12-01') AND date('2021-12-07')")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
query7.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------+----------+
|product_id|category_id|  date_utc|
+----------+-----------+----------+
|   4896069|       1660|2021-12-04|
| 200400482|       8694|2021-12-01|
| 201657258|       2520|2021-12-01|
| 201657258|       2520|2021-12-01|
| 201657258|       2520|2021-12-01|
| 200400482|       8694|2021-12-01|
| 200927671|       2800|2021-12-04|
| 200927671|       2800|2021-12-04|
| 200927671|       2800|2021-12-04|
| 200927671|       2800|2021-12-04|
| 200927671|       2800|2021-12-04|
| 200927671|       2800|2021-12-04|
| 200927671|       2800|2021-12-04|
| 200927671|       2800|2021-12-04|
|   5211019|       9552|2021-12-01|
|   5211019|       9552|2021-12-01|
|   6961306|       9552|2021-12-01|
|   5211019|       9552|2021-12-01|
| 201660396|       6992|2021-12-01|
| 201617686|       6073|2021-12-06|
+----------+-----------+----------+
only showing top 20 rows

In [42]:
query7.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (13)
+- Project (12)
   +- SortMergeJoin Inner (11)
      :- Sort (5)
      :  +- Exchange (4)
      :     +- Project (3)
      :        +- Filter (2)
      :           +- Scan parquet default.bucketed_fact_leadouts_msid (1)
      +- Sort (10)
         +- Exchange (9)
            +- Project (8)
               +- Filter (7)
                  +- Scan parquet default.bucketed_fact_dk_earnings (6)


(1) Scan parquet default.bucketed_fact_leadouts_msid
Output [5]: [cookie_value#711, category_id#712, product_id#713, date_utc#758, hour#759]
Batched: true
Location: InMemoryFileIndex [s3://die-stg-733963962829-eu-central-1/linh_thesis/bucketed_fact_leadouts_msid/date_utc=2021-12-02/hour=10, ... 167 entries]
PartitionFilters: [isnotnull(date_utc#758), (date_utc#758 >= 18962), (date_utc#758 <= 18968), dynamicpruningexpression(true)]
PushedFilters: [IsNotNull(cookie_value)]
ReadSchema: struct<cookie_value:string,category_id:string,product_id:string>

(2) Filte

### f) Group-by query

In [43]:
query8 = spark.sql("SELECT cookie_value, count(leadout_click_bokey) as bokey_count \
                    FROM fact_dk_earnings_original \
                    WHERE date_utc BETWEEN date('2021-12-01') AND date('2021-12-07') \
                    GROUP BY cookie_value \
                    ORDER BY bokey_count desc")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
query8.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------+
|      cookie_value|bokey_count|
+------------------+-----------+
|016a9lf500kvjeuyv7|         58|
|01o2l67300kk8rmt5b|         50|
|01ujv9qe00kji9jf6q|         34|
|  5075c9804df31f78|         34|
|01ycw8z500kjwt2m3d|         28|
|01xflwwn00kwgl7c1a|         24|
|01fidcpp00kwdb3jge|         24|
|017dc0ev00k4i8cjzu|         22|
|016dffnw00kjkbid7z|         22|
|01mc4d6t00kj7qyos1|         21|
|01fxf1i200kocg0fld|         21|
|01inmd5k00kdr0pu54|         21|
|01pt893200klgdbfxu|         19|
|01u92sb32xkwqgcb9x|         17|
|0183igr300kwdffh4p|         16|
|01kh0h0500jlb0aqxu|         16|
|018u8c0a00kwn5v53b|         16|
|01tk5y3m00kwnc19rh|         15|
|01fuysrx00k8f1tltu|         14|
|01xy0h0300jr0tb0ab|         14|
+------------------+-----------+
only showing top 20 rows

In [45]:
query8.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (8)
+- Sort (7)
   +- Exchange (6)
      +- HashAggregate (5)
         +- Exchange (4)
            +- HashAggregate (3)
               +- Project (2)
                  +- Scan parquet  (1)


(1) Scan parquet 
Output [4]: [cookie_value#10, leadout_click_bokey#25, date_utc#28, hour#29]
Batched: true
Location: InMemoryFileIndex [s3://die-stg-733963962829-eu-central-1/linh_thesis/fact_dk_earnings]
PartitionFilters: [isnotnull(date_utc#28), (date_utc#28 >= 18962), (date_utc#28 <= 18968)]
ReadSchema: struct<cookie_value:string,leadout_click_bokey:string>

(2) Project
Output [2]: [cookie_value#10, leadout_click_bokey#25]
Input [4]: [cookie_value#10, leadout_click_bokey#25, date_utc#28, hour#29]

(3) HashAggregate
Input [2]: [cookie_value#10, leadout_click_bokey#25]
Keys [1]: [cookie_value#10]
Functions [1]: [partial_count(leadout_click_bokey#25)]
Aggregate Attributes [1]: [count#1242L]
Results [2]: [cookie_value#10, count#1243L]

(4) Exchange
Input [2]: [

In [46]:
query9 = spark.sql("SELECT cookie_value, count(leadout_click_bokey) as bokey_count \
                    FROM bucketed_fact_dk_earnings \
                    WHERE date_utc BETWEEN date('2021-12-01') AND date('2021-12-07') \
                    GROUP BY cookie_value \
                    ORDER BY bokey_count desc")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
query9.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------+
|      cookie_value|bokey_count|
+------------------+-----------+
|016a9lf500kvjeuyv7|         58|
|01o2l67300kk8rmt5b|         50|
|01ujv9qe00kji9jf6q|         34|
|  5075c9804df31f78|         34|
|01ycw8z500kjwt2m3d|         29|
|01xflwwn00kwgl7c1a|         24|
|01fidcpp00kwdb3jge|         24|
|016dffnw00kjkbid7z|         22|
|017dc0ev00k4i8cjzu|         22|
|01mc4d6t00kj7qyos1|         21|
|01fxf1i200kocg0fld|         21|
|01inmd5k00kdr0pu54|         21|
|01pt893200klgdbfxu|         19|
|01u92sb32xkwqgcb9x|         17|
|01kh0h0500jlb0aqxu|         16|
|0183igr300kwdffh4p|         16|
|018u8c0a00kwn5v53b|         16|
|01tk5y3m00kwnc19rh|         15|
|01fuysrx00k8f1tltu|         14|
|01xy0h0300jr0tb0ab|         14|
+------------------+-----------+
only showing top 20 rows

In [48]:
query9.explain(mode="formatted")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
AdaptiveSparkPlan (7)
+- Sort (6)
   +- Exchange (5)
      +- HashAggregate (4)
         +- HashAggregate (3)
            +- Project (2)
               +- Scan parquet default.bucketed_fact_dk_earnings (1)


(1) Scan parquet default.bucketed_fact_dk_earnings
Output [4]: [cookie_value#557, leadout_click_bokey#572, date_utc#575, hour#576]
Batched: true
Location: InMemoryFileIndex [s3://die-stg-733963962829-eu-central-1/linh_thesis/bucketed_fact_dk_earnings/date_utc=2021-12-01/hour=16, ... 167 entries]
PartitionFilters: [isnotnull(date_utc#575), (date_utc#575 >= 18962), (date_utc#575 <= 18968)]
ReadSchema: struct<cookie_value:string,leadout_click_bokey:string>
SelectedBucketsCount: 4 out of 4

(2) Project
Output [2]: [cookie_value#557, leadout_click_bokey#572]
Input [4]: [cookie_value#557, leadout_click_bokey#572, date_utc#575, hour#576]

(3) HashAggregate
Input [2]: [cookie_value#557, leadout_click_bokey#572]
Keys [1]: [cookie_value#557]
Functions [1]: [partial_count(